In [ ]:
%env OGDF_BUILD_DIR=~/Bachelorarbeit/OGDF/build-debug

from ogdf_python_widget import ogdf, cppinclude
from ogdf_python_widget.widget import Widget
import ogdf_python_widget
import ipywidgets as widgets
from ipywidgets import HBox, VBox

cppinclude("ogdf/basic/graph_generators/randomized.h")
cppinclude("ogdf/layered/SugiyamaLayout.h")

G = ogdf.Graph()
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)

s = G.newNode()
a = G.newNode()
b = G.newNode()
c = G.newNode()
d = G.newNode()
e = G.newNode()
f = G.newNode()
t = G.newNode()

s_a = G.newEdge(s,a)
s_b = G.newEdge(s,b)
s_f = G.newEdge(s,f)
a_b = G.newEdge(a,b)
a_c = G.newEdge(a,c)
a_d = G.newEdge(a,d)
b_c = G.newEdge(b,c)
c_f = G.newEdge(c,f)
c_e = G.newEdge(c,e)
c_t = G.newEdge(c,t)
d_b = G.newEdge(d,b)
d_e = G.newEdge(d,e)
d_t = G.newEdge(d,t)
e_t = G.newEdge(e,t)
f_t = G.newEdge(f,t)

height = ogdf.NodeArray[int](G)
excess = ogdf.NodeArray[int](G)
name = ogdf.NodeArray[str](G)

capacity = ogdf.EdgeArray[int](G)
flow = ogdf.EdgeArray[int](G)
reverse_flow = ogdf.EdgeArray[int](G)

def relabel_node(node):
    GA.label[node] = name[node] + "|" + str(height[node]) + "|" + str(excess[node])
    
def relabel_edge(edge):
    GA.label[edge] = str(capacity[edge]) + "|" + str(flow[edge]) + "|" + str(reverse_flow[edge])
    
def get_edge_from_nodes(source, target):
    for e in G.edges:
        if e.source().index() == source.index() and e.target().index() == target.index():
            return e
        

name[s] = "s"
name[a] = "a"
name[b] = "b"
name[c] = "c"
name[d] = "d"
name[e] = "e"
name[f] = "f"
name[t] = "t"

capacity[s_a] = 38
capacity[s_b] = 1
capacity[s_f] = 2
capacity[a_b] = 8
capacity[a_c] = 10
capacity[a_d] = 13
capacity[b_c] = 26
capacity[c_f] = 24
capacity[c_e] = 8
capacity[c_t] = 1
capacity[d_b] = 2
capacity[d_e] = 1
capacity[d_t] = 7
capacity[e_t] = 7
capacity[f_t] = 27

for n in G.nodes:
    height[n] = 0
    excess[n] = 0
    relabel_node(n)
    
for e in G.edges:
    flow[e] = 0
    reverse_flow[e] = 0
    relabel_edge(e)
    
height[s] = 8
excess[s] = 1000
SL = ogdf.SugiyamaLayout()
SL.call(GA)

w = Widget(GA, True)

def push(edge, reversePush = False):
    if(reversePush):
        if height[edge.target()] <= height[edge.source()]:
            return
        send = min(excess[edge.target()], capacity[edge] - reverse_flow[edge]) * (-1)
    else: 
        if height[edge.source()] <= height[edge.target()]:
            return
        send = min(excess[edge.source()], capacity[edge] - flow[edge])
        
    flow[edge] += send
    reverse_flow[edge] -= send
    excess[edge.source()] -= send
    excess[edge.target()] += send
    relabel_node(edge.source())
    relabel_node(edge.target())
    w.update_node(edge.source())
    w.update_node(edge.target())
    relabel_edge(edge)
    w.update_link(edge)
    
def relabel(u):
    # Find smallest new height making a push possible,
    # if such a push is possible at all.
    min_height = 1000000
    for v in G.nodes:
        edge = get_edge_from_nodes(u,v)
        reverseEdge = get_edge_from_nodes(v,u)
        if edge is not None and capacity[edge] - flow[edge] > 0:
            min_height = min(min_height, height[v])
            height[u] = min_height + 1
            relabel_node(u)
            w.update_node(u)
        elif reverseEdge is not None and capacity[reverseEdge] - reverse_flow[reverseEdge] > 0:
            min_height = min(min_height, height[v])
            height[u] = min_height + 1
            relabel_node(u)
            w.update_node(u)

selectedLink = None

def clickLink(link, alt, ctrl):
    if not alt:
        push(link, ctrl)
    if alt:
        #select link
        if selectedLink is not None:
            #unselect currently selected
            w.graph_attributes.strokeColor[selectedLink] = ogdf.Color(0, 0, 0)
            w.update_link(selectedLink)
        w.graph_attributes.strokeColor[link] = ogdf.Color(150, 231, 255)
        w.update_link(link)
        update_selected_link(link)
        
def clickSvg(x, y, alt, ctrl, backgroundClicked):
    global w, selectedLink
    if backgroundClicked and not ctrl and not alt:
        #unselect link
        if selectedLink is not None:
            w.graph_attributes.strokeColor[selectedLink] = ogdf.Color(0, 0, 0)
            w.update_link(selectedLink)
            update_selected_link(None)
            
w.on_link_click_callback = clickLink
w.on_node_click_callback = lambda node, alt, ctrl: relabel(node)
w.on_svg_click_callback = clickSvg
    
def update_selected_link(link):
    global selectedLink
    selectedLink = link
    update_link_texts()

##################################################################################

def create_text_widget(name, disabled = False):
    return widgets.Text(
    value='',
    placeholder='-',
    description=name,
    disabled=disabled
)

################################################################################## 

def update_link_texts():
    global link_id_text, selectedLink
    if selectedLink is not None:
        link_capacity_text.value = str(capacity[selectedLink])
        link_flow_text.value = str(flow[selectedLink])
        link_reverse_flow_text.value = str(reverse_flow[selectedLink])
    else:
        link_capacity_text.value = ''
        link_flow_text.value = ''
        link_reverse_flow_text.value = ''

link_capacity_text = create_text_widget('capacity:')
link_flow_text = create_text_widget('flow:', True)
link_reverse_flow_text = create_text_widget('reverse flow:', True)

def on_save_button_clicked(b):
    if selectedLink is not None and link_capacity_text.value.isdigit():
        capacity[selectedLink] = int(link_capacity_text.value)
        relabel_edge(selectedLink)
        w.update_link(selectedLink)
        
save_button = widgets.Button(description="Save")
save_button.on_click(on_save_button_clicked)

##################################################################################

link_texts_vbox = VBox([link_capacity_text, link_flow_text, link_reverse_flow_text, save_button])

vbox = VBox([link_texts_vbox, w])

vbox

In [ ]:
push(s_a)
push(s_b)
push(s_f)